In [ ]:
!pip install transformers --quiet
!pip install -q huggingface_hub
!pip install -U datasets


In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model=BertModel.from_pretrained('bert-base-uncased',output_hidden_states=True,output_attentions=True)

inputs=tokenizer("Hello, how are you?",return_tensors='pt')
outputs=model(**inputs)
last_hidden_states=outputs.last_hidden_state
pooler_output=outputs.pooler_output
attentions=outputs.attentions
hidden_states=outputs.hidden_states


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


last hidden state is used for sequence label tagging like  NER


pooler output is used for sentence categorization like text classification

In [ ]:
last_hidden_states

tensor([[[-0.0824,  0.0667, -0.2880,  ..., -0.3566,  0.1960,  0.5381],
         [ 0.0310, -0.1448,  0.0952,  ..., -0.1560,  1.0151,  0.0947],
         [-0.8935,  0.3240,  0.4184,  ..., -0.5498,  0.2853,  0.1149],
         ...,
         [-0.2812, -0.8531,  0.6912,  ..., -0.5051,  0.4716, -0.6854],
         [-0.4429, -0.7820, -0.8055,  ...,  0.1949,  0.1081,  0.0130],
         [ 0.5570, -0.1080, -0.2412,  ...,  0.2817, -0.3996, -0.1882]]],
       grad_fn=<NativeLayerNormBackward0>)

In [ ]:
pooler_output


tensor([[-0.9397, -0.4081, -0.9024,  0.8667,  0.6076, -0.1782,  0.9319,  0.2685,
         -0.7918, -1.0000, -0.4899,  0.9625,  0.9823,  0.6102,  0.9614, -0.8728,
         -0.6449, -0.6543,  0.3102, -0.6648,  0.7556,  1.0000,  0.0778,  0.3350,
          0.5094,  0.9948, -0.8847,  0.9590,  0.9761,  0.7384, -0.7787,  0.1232,
         -0.9912, -0.2119, -0.9225, -0.9931,  0.3767, -0.8050, -0.0945,  0.0497,
         -0.9269,  0.2934,  1.0000,  0.0954,  0.4904, -0.3206, -1.0000,  0.2759,
         -0.9303,  0.9396,  0.9162,  0.8490,  0.1967,  0.4833,  0.5391,  0.0523,
         -0.0621,  0.1046, -0.3403, -0.6018, -0.6203,  0.5439, -0.8909, -0.9180,
          0.9392,  0.8156, -0.1900, -0.2589, -0.1371, -0.1553,  0.9367,  0.3183,
          0.2468, -0.9167,  0.5823,  0.2861, -0.7534,  1.0000, -0.7911, -0.9841,
          0.8615,  0.7741,  0.6616, -0.2575,  0.4217, -1.0000,  0.6669, -0.1115,
         -0.9931,  0.1993,  0.6389, -0.3458,  0.3185,  0.6436, -0.6661, -0.4977,
         -0.3677, -0.8909, -

In [ ]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

model_name="textattack/bert-base-uncased-SST-2"
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
text="I absolutely dont like that movie.It was worst!"

inputs=tokenizer(text,return_tensors='pt')
inputs

{'input_ids': tensor([[ 101, 1045, 7078, 2123, 2102, 2066, 2008, 3185, 1012, 2009, 2001, 5409,
          999,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
with torch.no_grad():
  outputs=model(**inputs)
  logits=outputs.logits
  probs=F.softmax(logits,dim=1)
  predicted_class=torch.argmax(probs,dim=1).item()



In [ ]:
labels=model.config.id2label
print(f"Predicted class: {labels[predicted_class]} prob={probs[0][predicted_class]:.4f}")

Predicted class: LABEL_0 prob=0.9991


In [ ]:
!pip install transformers tensorflow --quiet

In [ ]:
import tensorflow as tf
from transformers import TFBertModel,BertTokenizer

class BertClassifier(tf.keras.Model):
  def __init__(self,num_name='bert-base-uncased',num_classes=2):
    super(BertClassifier,self).__init__()
    self.bert=TFBertModel.from_pretrained(model_name, from_pt=True)
    self.dropout=tf.keras.layers.Dropout(0.3)
    self.classifier=tf.keras.layers.Dense(num_classes,activation='softmax')


  def call(self,inputs,training=False):
    outputs=self.bert(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        token_type_ids=inputs['token_type_ids'] if 'token_type_ids' in inputs else None,
        training=training
    )

    cls_output=outputs.pooler_output
    x=self.dropout(cls_output,training=training)
    return self.classifier(x)

In [ ]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
text="I really wonna visit your home town!!!"

encoded_input=tokenizer(text,return_tensors='tf',padding=True,truncation=True)


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [ ]:
model=BertClassifier(num_classes=2)

model.trainable=False

logits=model(encoded_input,training=False)
predicted_class=tf.argmax(logits,axis=1).numpy()[0]
confidence=tf.reduce_max(logits,axis=1).numpy()[0]

print(f"Predicted class:{predicted_class},confidence:{confidence:4f}")

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Predicted class:1,confidence:0.596933
